# Data Preprocessing

## Importing Libraries

We will start by importing the necessary libraries for this notebook.

In [12]:
import pandas as pd
import numpy as np
import category_encoders as ce
from scipy.signal import ShortTimeFFT
from sklearn.preprocessing import OrdinalEncoder,TargetEncoder,LabelEncoder
from src.preprocessing import *


## Loading Data

In [13]:
df = cargar_datos('../data/raw/application_train.csv')
desc = forma(df)
desc


'307511 filas y 122 columnas'

Let's see the first rows of the dataset.

In [14]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


We are going to store the target conlumn in a separate variable.

In [15]:
y = df["TARGET"]
df = df.drop("TARGET", axis=1)

We are going to convert SK_ID_CURR to string and store it in a separate variable.

In [16]:
df["SK_ID_CURR"] = df["SK_ID_CURR"].astype(str)
sk_id_curr = df["SK_ID_CURR"]
df = df.drop("SK_ID_CURR", axis=1)

We are going to start converting wrong formatted boolean columns to the correct format.

In [17]:
y_n_cols = ["FLAG_OWN_CAR","FLAG_OWN_REALTY"]
df = format_boolean_columns(df,y_n_cols,true_label="Y",false_label="N")
df = format_boolean_columns(df,"EMERGENCYSTATE_MODE",true_label="Yes",false_label="No")
cols_1_0 = ["FLAG_MOBIL","FLAG_EMP_PHONE","FLAG_WORK_PHONE","FLAG_CONT_MOBILE","FLAG_PHONE","FLAG_EMAIL","REG_REGION_NOT_LIVE_REGION","REG_REGION_NOT_WORK_REGION","LIVE_REGION_NOT_WORK_REGION","REG_CITY_NOT_LIVE_CITY","REG_CITY_NOT_WORK_CITY","LIVE_CITY_NOT_WORK_CITY","FLAG_DOCUMENT_2", "FLAG_DOCUMENT_3", "FLAG_DOCUMENT_4", "FLAG_DOCUMENT_5", "FLAG_DOCUMENT_6", "FLAG_DOCUMENT_7", "FLAG_DOCUMENT_8", "FLAG_DOCUMENT_9", "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11", "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16", "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19", "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21"]
df[cols_1_0] = df[cols_1_0].astype(bool)

We are going to explore the missing values percentages of the columns.

In [18]:
missing_values_perc = missing_values_percentage(df)
missing_values_perc

,Missing Percentage
COMMONAREA_MEDI,69.872297
COMMONAREA_MODE,69.872297
COMMONAREA_AVG,69.872297
NONLIVINGAPARTMENTS_MODE,69.432963
NONLIVINGAPARTMENTS_MEDI,69.432963
...,...
EXT_SOURCE_2,0.214626
AMT_GOODS_PRICE,0.090403
AMT_ANNUITY,0.003902
CNT_FAM_MEMBERS,0.000650


We are going to drop the columns with more than 45% of missing values.

In [19]:
df = drop_columns(df,missing_values_perc,45)
missing_values_perc = missing_values_percentage(df)

We are going to check the remaining columns in df.

In [20]:
forma(df)

'307511 filas y 71 columnas'

Let's check the columns with missing values.

In [21]:
df[missing_values_perc.index]

,OCCUPATION_TYPE,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,NAME_TYPE_SUITE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE
0,Laborers,0.139376,0.0,0.0,1.0,0.0,0.0,0.0,Unaccompanied,2.0,2.0,2.0,2.0,0.262949,351000.0,24700.5,1.0,-1134.0
1,Core staff,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Family,1.0,0.0,0.0,1.0,0.622246,1129500.0,35698.5,2.0,-828.0
2,Laborers,0.729567,0.0,0.0,0.0,0.0,0.0,0.0,Unaccompanied,0.0,0.0,0.0,0.0,0.555912,135000.0,6750.0,1.0,-815.0
3,Laborers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unaccompanied,2.0,0.0,0.0,2.0,0.650442,297000.0,29686.5,2.0,-617.0
4,Core staff,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Unaccompanied,0.0,0.0,0.0,0.0,0.322738,513000.0,21865.5,1.0,-1106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,Sales staff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unaccompanied,0.0,0.0,0.0,0.0,0.681632,225000.0,27558.0,1.0,-273.0
307507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unaccompanied,0.0,0.0,0.0,0.0,0.115992,225000.0,12001.5,1.0,0.0
307508,Managers,0.218859,0.0,1.0,1.0,0.0,1.0,0.0,Unaccompanied,6.0,0.0,0.0,6.0,0.535722,585000.0,29979.0,1.0,-1909.0
307509,Laborers,0.661024,0.0,0.0,0.0,0.0,0.0,0.0,Unaccompanied,0.0,0.0,0.0,0.0,0.514163,319500.0,20205.0,2.0,-322.0


We are going to fill missing values with 0s in the following columns: 

AMT_REQ_CREDIT_BUREAU_HOUR, AMT_REQ_CREDIT_BUREAU_DAY, AMT_REQ_CREDIT_BUREAU_WEEK, AMT_REQ_CREDIT_BUREAU_MON, AMT_REQ_CREDIT_BUREAU_QRT, AMT_REQ_CREDIT_BUREAU_YEAR

This decision is made because these columns are related to the number of enquiries to the Credit Bureau and it is possible that the missing values are due to the fact that the client has not made any requests.

In [22]:
amt_req_credit_bureau_cols = ["AMT_REQ_CREDIT_BUREAU_HOUR","AMT_REQ_CREDIT_BUREAU_DAY","AMT_REQ_CREDIT_BUREAU_WEEK","AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_YEAR"]
df[amt_req_credit_bureau_cols] = df[amt_req_credit_bureau_cols].fillna(0)

We are going to also fill missing values with the mode in the following columns:

OBS_30_CNT_SOCIAL_CIRCLE, DEF_30_CNT_SOCIAL_CIRCLE, OBS_60_CNT_SOCIAL_CIRCLE, DEF_60_CNT_SOCIAL_CIRCLE

This decision is made because these columns are related to the number of observations of clients' social surroundings and it is possible that the missing values are due to the fact that the client has not made any observations.

In [23]:
social_circle_cols = ["OBS_30_CNT_SOCIAL_CIRCLE","DEF_30_CNT_SOCIAL_CIRCLE","OBS_60_CNT_SOCIAL_CIRCLE","DEF_60_CNT_SOCIAL_CIRCLE"]
df[social_circle_cols] = df[social_circle_cols].fillna(0)

We are going to fill missing values in the column CNT_FAM_MEMBERS with 1. This decision is made because if there is not data about the number of family members, we can assume that the client is alone. 

In [24]:
df["CNT_FAM_MEMBERS"] = df["CNT_FAM_MEMBERS"].fillna(1)

We are going to fill missing values in the columns NAME_TYPE_SUITE with unaccompanied. This decision is made because if there is not data about wwho is accompanying the client, we can assume that the client is unaccompanied.

In [25]:
df["NAME_TYPE_SUITE"] = df["NAME_TYPE_SUITE"].fillna("Unaccompanied")

In [26]:
df.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,False,True,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,False,False,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,True,True,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0
3,Cash loans,F,False,True,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0
4,Cash loans,M,False,True,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0


In order to use the imputers, we need to encode the categorical columns.

One Hot Encoding for NAME_CONTRACT_TYPE and CODE_GENDER columns, as they only have two possible values, so we are not going to increase our dimensionality heavily.

In [27]:
#Delete Rows with CODE_GENDER = XNA, only 4 in all dataset
y = y[df['CODE_GENDER'] != "XNA"]
sk_id_curr = sk_id_curr[df['CODE_GENDER'] != "XNA"]
df = df[df['CODE_GENDER'] != "XNA"]

one_hot_encoding_columns = ["NAME_CONTRACT_TYPE","CODE_GENDER"]
df = pd.get_dummies(df,columns=one_hot_encoding_columns ,drop_first=True)

Categorical Encoding for NAME_EDUCATION_TYPE and WEEKDAY_APPR_PROCESS_START columns.

In [28]:
education_order = ['Lower secondary', 'Secondary / secondary special', 'Incomplete higher', 
                   'Higher education', 'Academic degree']
encoderOrdinalEdu = OrdinalEncoder(categories=[education_order])
df['NAME_EDUCATION_TYPE'] = encoderOrdinalEdu.fit_transform(df[['NAME_EDUCATION_TYPE']])

week_order = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']
encoderOrdinalWeek = OrdinalEncoder(categories=[week_order])
df['WEEKDAY_APPR_PROCESS_START'] = encoderOrdinalWeek.fit_transform(df[['WEEKDAY_APPR_PROCESS_START']])

Target Encoding for the rest of the categorical columns, without OCCUPATION_TYPE.

In [29]:
# target_encoding_columns = ["NAME_TYPE_SUITE","NAME_INCOME_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","ORGANIZATION_TYPE"]
#encoderTarget = TargetEncoder()
#df[target_encoding_columns] = encoderTarget.fit_transform(df[target_encoding_columns], y) 

In [30]:
label_encoding_columns = [
    "NAME_TYPE_SUITE",
    "NAME_INCOME_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "ORGANIZATION_TYPE"
]

# A dictionary to store the encoders
encoders = {}

# Encoding each column
for column in label_encoding_columns:
    encoder = LabelEncoder()
    df[column] = encoder.fit_transform(df[column])
    encoders[column] = encoder 

We are going to enconde OCCUPATION_TYPE using the One Hot Encoding.

In [31]:
df = pd.get_dummies(df,columns=["OCCUPATION_TYPE"],drop_first=True)

In [32]:
df_imputed = impute_with_knn(df, ['OCCUPATION_TYPE', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'AMT_ANNUITY',
                          'DAYS_LAST_PHONE_CHANGE'])

KeyboardInterrupt: 

Saving the numerical imputed df.

In [4]:
df_imputed.to_csv('../data/processed/num_imputed_df.csv', index=False) 

NameError: name 'df_imputed' is not defined

We now are going to decode the columns that were encoded.

First, we copy the dataframe.

In [47]:
df_imp_imported = pd.read_csv('../data/processed/num_imputed_df.csv') #Read the data after imputing to avoid loosing time.
df_cat = df_imp_imported.copy()

In [29]:
df_cat = df_imputed.copy()

We decode OCCUPATION_TYPE.

In [48]:
occupation_columns = df_cat.columns[df_cat.columns.str.startswith('OCCUPATION_TYPE_')]
df_cat['OCCUPATION_TYPE'] = df_cat[occupation_columns].idxmax(axis=1).str.replace('OCCUPATION_TYPE_', '')
df_cat = df_cat.drop(columns=occupation_columns)

We decode NAME_EDUCATION_TYPE using the inverse_transform function of the encoder.

In [49]:
df_cat['NAME_EDUCATION_TYPE'].head()

0    1.0
1    3.0
2    1.0
3    1.0
4    1.0
Name: NAME_EDUCATION_TYPE, dtype: float64

In [40]:
df_cat['WEEKDAY_APPR_PROCESS_START'] = encoderOrdinalWeek.inverse_transform(df_cat[['WEEKDAY_APPR_PROCESS_START']]).ravel()

df_cat['WEEKDAY_APPR_PROCESS_START']

ValueError: could not convert string to float: 'WEDNESDAY'

In [51]:
# Decoding NAME_EDUCATION_TYPE
df_cat['NAME_EDUCATION_TYPE'] = encoderOrdinalEdu.inverse_transform(df_cat[['NAME_EDUCATION_TYPE']]).ravel()

In [52]:
df_cat['NAME_EDUCATION_TYPE'].head()

0    Secondary / secondary special
1                 Higher education
2    Secondary / secondary special
3    Secondary / secondary special
4    Secondary / secondary special
Name: NAME_EDUCATION_TYPE, dtype: object

 We decode CODE_GENDER mapping 1s in CODE_GENDER_M into M and 0s into F.

In [36]:
# Decoding CODE_GENDER
df_cat['CODE_GENDER'] = df_cat['CODE_GENDER_M'].map({1: 'M', 0: 'F'})

In [43]:
df_cat['CODE_GENDER'].head()

0    M
1    F
2    M
3    F
4    M
Name: CODE_GENDER, dtype: object

We decode NAME_CONTRACT_TYPE mapping 1s in column NAME_CONTRACT_TYPE_Revolving loans into Revolving loans and 0s into cash loans.

In [37]:
# Decoding NAME_CONTRACT_TYPE
df_cat['NAME_CONTRACT_TYPE'] = df_cat['NAME_CONTRACT_TYPE_Revolving loans'].map({1: 'Revolving loans', 0: 'Cash loans'})

In [44]:
df_cat['NAME_CONTRACT_TYPE'].head()

0         Cash loans
1         Cash loans
2    Revolving loans
3         Cash loans
4         Cash loans
Name: NAME_CONTRACT_TYPE, dtype: object

We decode WEEKDAY_APPR_PROCESS_START using the inverse_transform function of the encoder.

In [38]:
# We transform the column values into integers since they are in float64
df_cat['WEEKDAY_APPR_PROCESS_START'] = df_cat['WEEKDAY_APPR_PROCESS_START'].astype(int)

In [40]:
df_cat['WEEKDAY_APPR_PROCESS_START'] = encoderOrdinalWeek.inverse_transform(df_cat[['WEEKDAY_APPR_PROCESS_START']]).ravel()

ValueError: could not convert string to float: 'WEDNESDAY'

In [42]:
df_cat['WEEKDAY_APPR_PROCESS_START'].head()

0    WEDNESDAY
1       MONDAY
2       MONDAY
3    WEDNESDAY
4     THURSDAY
Name: WEEKDAY_APPR_PROCESS_START, dtype: object